In [7]:
from __future__ import print_function

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

# Import MNIST data
import load_data as data

In [8]:
# n different points we want to cluster
# We have 59 points in all 3 categories * 2 people * 10 repetitions (maia_fall only had 9)
train_trials = 59
test_trials = 16
# Tensor length of 80 since that's how many frames we decided on--easily modifiable
NUM_FRAMES = 80
NUM_FEATURES = 36
NUM_CLASSES = 6


In [10]:
#### Use Weighting Method to Refine Training
# Based on the information about which joints most accurately classify

# 2D
xtr = tf.placeholder("float", [train_trials, NUM_FRAMES])
xte = tf.placeholder("float", [NUM_FRAMES])

# Nearest Neighbor calculation using L1 Distance
# Calculate L1 Distance
distance = tf.reduce_sum(tf.abs(tf.add(xtr, tf.negative(xte))), reduction_indices=1)
# Prediction: Get min distance index (Nearest neighbor)
pred = tf.argmin(distance, 0)
init = tf.global_variables_initializer()

label_votes = np.zeros([test_trials, NUM_CLASSES])
accuracies_for_tests = np.zeros(test_trials)

best_features_for_l1 = [6, 12, 14]


for i in range(test_trials):
    for j in range(NUM_FEATURES):
#     for j in best_features_for_l1:
    # for each test example, classify with one keypoint at a time -- 
    # each keypoint will vote for the label it predicts
        ###### Training code
        Xtr, Ytr = data.train_get_one_keypoint(train_trials, j)
        Xte, Yte = data.test_get_one_keypoint(test_trials, j)

        with tf.Session() as sess:
            sess.run(init)
            #2D
            nn_index = sess.run(pred, feed_dict={xtr: Xtr, xte: Xte[i, :]})
            predicted = np.argmax(Ytr[nn_index]) 
            label_votes[i, predicted] = label_votes[i, predicted] + 1
#             print(label_votes)

# End testing, now calculate accuracy given labels
accuracy = 0.
for i in range(test_trials):
    predicted = np.argmax(label_votes[i, :])
    if predicted == np.argmax(Yte[i]):
        accuracy += 1./len(Xte)
    print("Test", i, "Prediction:", predicted + 1, \
    "True Class:", np.argmax(Yte[i]) + 1)   

print(accuracy)


Test 0 Prediction: 5 True Class: 1
Test 1 Prediction: 6 True Class: 2
Test 2 Prediction: 2 True Class: 6
Test 3 Prediction: 2 True Class: 2
Test 4 Prediction: 2 True Class: 2
Test 5 Prediction: 3 True Class: 3
Test 6 Prediction: 4 True Class: 1
Test 7 Prediction: 1 True Class: 1
Test 8 Prediction: 2 True Class: 5
Test 9 Prediction: 2 True Class: 5
Test 10 Prediction: 5 True Class: 1
Test 11 Prediction: 3 True Class: 3
Test 12 Prediction: 2 True Class: 2
Test 13 Prediction: 6 True Class: 6
Test 14 Prediction: 5 True Class: 1
Test 15 Prediction: 6 True Class: 1
0.4375
